### requirements for the following codings


In [1]:
### packages required 
!pip install fair-esm 
!pip install torch
!pip install tensorflow
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 93 kB 1.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1309 sha256=44722408e130d12025d0e667d0fa527d472b96ce512e84f513303239f4de7b19
  Stored in directory: /root/.cache/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn


### peptide embeddings with esm2_t6_8M_UR50D pretrained models
6 layers, 8M parameters, dataset: UR50/D 2021_04, embedding dimension: 320
mode download URL: https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t6_8M_UR50D.pt

In [2]:
def esm_embeddings(peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long, 
  #         or you have too many sequences for transformation in a single converting, 
  #         you conputer might automatically kill the job.
  import torch
  import esm
  import collections
  # load the model
  # NOTICE: if the model was not downloaded in your local environment, it will automatically download it.
  model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
  batch_converter = alphabet.get_batch_converter()
  model.eval()  # disables dropout for deterministic results

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t6_8M_UR50D' only has 6 layers, and therefore repr_layers parameters is equal to 6
      results = model(batch_tokens, repr_layers=[6], return_contacts=True)  
  token_representations = results["representations"][6]

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  # save dataset
  # sequence_representations is a list and each element is a tensor
  embeddings_results = collections.defaultdict(list)
  for i in range(len(sequence_representations)):
      # tensor can be transformed as numpy sequence_representations[0].numpy() or sequence_representations[0].to_list
      each_seq_rep = sequence_representations[i].tolist()
      for each_element in each_seq_rep:
          embeddings_results[i].append(each_element)
  embeddings_results = pd.DataFrame(embeddings_results).T
  return embeddings_results

### data loading and embeddings

In [4]:
import numpy as np
import pandas as pd

In [5]:
# training dataset loading
dataset = pd.read_excel('SCMRSA_train.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 

embeddings_results = pd.DataFrame()
for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings(peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])

embeddings_results.to_csv('SCMRSA_train_esm2_t6_8M_UR50D_unified_320_dimension.csv')

# loading the y dataset for model development 
y_train = dataset['label'] 
y_train = np.array(y_train) # transformed as np.array for CNN model

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t6_8M_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t6_8M_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D-contact-regression.pt


In [6]:
# test dataset loading
dataset = pd.read_excel('SCMRSA_test.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 
embeddings_results = pd.DataFrame()
# embedding all the peptide one by one
for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings(peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])

embeddings_results.to_csv('SCMRSA_test_esm2_t6_8M_UR50D_unified_320_dimension.csv')


# loading the y dataset for model development 
y_test = dataset['label']
y_test = np.array(y_test) # transformed as np.array for CNN model

In [7]:
# assign the dataset 
X_train_data_name = 'SCMRSA_train_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_train_data = pd.read_csv(X_train_data_name,header=0, index_col = 0,delimiter=',')

X_test_data_name = 'SCMRSA_test_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_test_data = pd.read_csv(X_test_data_name,header=0, index_col = 0,delimiter=',')

X_train = np.array(X_train_data)
X_test = np.array(X_test_data)

# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range 
X_test = scaler.transform(X_test)

In [8]:
# check the dimension of the dataset before model development
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(796, 320)
(199, 320)
(796,)
(199,)


### Model architecture

In [9]:
def ESM_CNN(X_train, y_train, X_test, y_test):
  from keras.layers import Input,InputLayer, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D,Conv1D
  from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, AveragePooling1D, MaxPooling1D
  from keras.models import Sequential,Model
  from keras.optimizers import SGD
  from keras.callbacks import ModelCheckpoint,LearningRateScheduler, EarlyStopping
  import keras
  from keras import backend as K
  inputShape=(320,1)
  input = Input(inputShape)
  x = Conv1D(128,(3),strides = (1),name='layer_conv1',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool1',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Conv1D(32,(3),strides = (1),name='layer_conv2',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Flatten()(x)
  x = Dense(64,activation = 'relu',name='fc1')(x)
  x = Dropout(0.15)(x)
  x = Dense(2,activation = 'softmax',name='fc2')(x)
  model = Model(inputs = input,outputs = x,name='Predict')
  # define SGD optimizer
  momentum = 0.5
  sgd = SGD(lr=0.01, momentum=momentum, decay=0.0, nesterov=False)
  # compile the model
  model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
  # learning deccay setting
  import math
  def step_decay(epoch): # gradually decrease the learning rate 
      initial_lrate=0.1
      drop=0.6
      epochs_drop = 3.0
      lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
            math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
      return lrate
  lrate = LearningRateScheduler(step_decay)

  # early stop setting
  early_stop = EarlyStopping(monitor='val_accuracy', patience = 40,restore_best_weights = True)

  # summary the callbacks_list
  callbacks_list = [ lrate , early_stop]

  model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                            epochs=200,callbacks=callbacks_list,batch_size = 8, verbose=1)
  return model, model_history

### 10-fold cross validation

In [10]:
#Implementing 10-fold cross validation
from sklearn.model_selection import KFold
k = 10 
kf = KFold(n_splits=k, shuffle = True, random_state=1)
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)

# result collection list
ACC_collecton = []
BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []

for train_index , test_index in kf.split(y_train):
    X_train_CV , X_valid_CV = X_train.iloc[train_index,:],X_train.iloc[test_index,:]
    y_train_CV , y_valid_CV = y_train.iloc[train_index] , y_train.iloc[test_index]
    model, model_history = ESM_CNN(X_train_CV, y_train_CV, X_valid_CV, y_valid_CV)
    # confusion matrix 
    predicted_class= []
    predicted_protability = model.predict(X_valid_CV,batch_size=1)
    for i in range(predicted_protability.shape[0]):
      index = np.where(predicted_protability[i] == np.amax(predicted_protability[i]))[0][0]
      predicted_class.append(index)
    predicted_class = np.array(predicted_class)
    y_true = y_valid_CV    
    from sklearn.metrics import confusion_matrix
    import math
    # np.ravel() return a flatten 1D array
    TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
    ACC = (TP+TN)/(TP+TN+FP+FN)
    ACC_collecton.append(ACC)
    Sn_collecton.append(TP/(TP+FN))
    Sp_collecton.append(TN/(TN+FP))
    MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
    MCC_collecton.append(MCC)
    BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
    from sklearn.metrics import roc_auc_score
    AUC = roc_auc_score(y_valid_CV, predicted_protability[:,1])
    AUC_collecton.append(AUC)


Epoch 1/200


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


90/90 [==============================] - 2s 8ms/step - loss: 1.8691 - accuracy: 0.8994 - val_loss: 0.3580 - val_accuracy: 0.8750 - lr: 0.1000
Epoch 2/200
90/90 [==============================] - 1s 6ms/step - loss: 0.1315 - accuracy: 0.9581 - val_loss: 0.1943 - val_accuracy: 0.9250 - lr: 0.1000
Epoch 3/200
90/90 [==============================] - 1s 6ms/step - loss: 0.0690 - accuracy: 0.9791 - val_loss: 0.1327 - val_accuracy: 0.9500 - lr: 0.0600
Epoch 4/200
90/90 [==============================] - 1s 7ms/step - loss: 0.0690 - accuracy: 0.9763 - val_loss: 0.0967 - val_accuracy: 0.9750 - lr: 0.0600
Epoch 5/200
90/90 [==============================] - 1s 6ms/step - loss: 0.0476 - accuracy: 0.9888 - val_loss: 0.1109 - val_accuracy: 0.9625 - lr: 0.0600
Epoch 6/200
90/90 [==============================] - 1s 6ms/step - loss: 0.0408 - accuracy: 0.9902 - val_loss: 0.1249 - val_accuracy: 0.9625 - lr: 0.0360
Epoch 7/200
90/90 [==============================] - 1s 6ms/step - loss: 0.0370 - accura

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


90/90 [==============================] - 1s 8ms/step - loss: 1.3669 - accuracy: 0.9050 - val_loss: 0.4071 - val_accuracy: 0.8750 - lr: 0.1000
Epoch 2/200
90/90 [==============================] - 1s 6ms/step - loss: 0.1399 - accuracy: 0.9539 - val_loss: 0.2830 - val_accuracy: 0.8750 - lr: 0.1000
Epoch 3/200
90/90 [==============================] - 1s 6ms/step - loss: 0.0987 - accuracy: 0.9721 - val_loss: 0.1657 - val_accuracy: 0.9375 - lr: 0.0600
Epoch 4/200
90/90 [==============================] - 1s 6ms/step - loss: 0.0757 - accuracy: 0.9777 - val_loss: 0.1701 - val_accuracy: 0.9375 - lr: 0.0600
Epoch 5/200
90/90 [==============================] - 1s 6ms/step - loss: 0.0701 - accuracy: 0.9777 - val_loss: 0.2021 - val_accuracy: 0.9375 - lr: 0.0600
Epoch 6/200
90/90 [==============================] - 1s 6ms/step - loss: 0.0458 - accuracy: 0.9832 - val_loss: 0.2280 - val_accuracy: 0.9500 - lr: 0.0360
Epoch 7/200
90/90 [==============================] - 1s 6ms/step - loss: 0.0436 - accura

...

90/90 [==============================] - 1s 8ms/step - loss: 0.7015 - accuracy: 0.8815 - val_loss: 0.6584 - val_accuracy: 0.8861 - lr: 0.1000
Epoch 2/200
90/90 [==============================] - 1s 6ms/step - loss: 0.1307 - accuracy: 0.9554 - val_loss: 0.6860 - val_accuracy: 0.4051 - lr: 0.1000
Epoch 3/200
90/90 [==============================] - 1s 6ms/step - loss: 0.0855 - accuracy: 0.9805 - val_loss: 0.5955 - val_accuracy: 0.9747 - lr: 0.0600
Epoch 4/200
90/90 [==============================] - 1s 6ms/step - loss: 0.0620 - accuracy: 0.9791 - val_loss: 0.3210 - val_accuracy: 0.9620 - lr: 0.0600
Epoch 5/200
90/90 [==============================] - 1s 6ms/step - loss: 0.0527 - accuracy: 0.9819 - val_loss: 0.2514 - val_accuracy: 0.9620 - lr: 0.0600
Epoch 6/200
90/90 [==============================] - 1s 6ms/step - loss: 0.0380 - accuracy: 0.9888 - val_loss: 0.2823 - val_accuracy: 0.9494 - lr: 0.0360
Epoch 7/200
90/90 [==============================] - 1s 7ms/step - loss: 0.0393 - accura

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


90/90 [==============================] - 1s 8ms/step - loss: 1.3086 - accuracy: 0.8619 - val_loss: 0.2750 - val_accuracy: 0.9114 - lr: 0.1000
Epoch 2/200
90/90 [==============================] - 1s 6ms/step - loss: 0.1831 - accuracy: 0.9498 - val_loss: 0.2569 - val_accuracy: 0.9114 - lr: 0.1000
Epoch 3/200
90/90 [==============================] - 1s 6ms/step - loss: 0.1019 - accuracy: 0.9721 - val_loss: 0.1982 - val_accuracy: 0.9494 - lr: 0.0600
Epoch 4/200
90/90 [==============================] - 1s 6ms/step - loss: 0.0826 - accuracy: 0.9763 - val_loss: 0.1380 - val_accuracy: 0.9494 - lr: 0.0600
Epoch 5/200
90/90 [==============================] - 1s 6ms/step - loss: 0.0667 - accuracy: 0.9777 - val_loss: 0.1226 - val_accuracy: 0.9494 - lr: 0.0600
Epoch 6/200
90/90 [==============================] - 1s 6ms/step - loss: 0.0552 - accuracy: 0.9805 - val_loss: 0.1058 - val_accuracy: 0.9747 - lr: 0.0360
Epoch 7/200
90/90 [==============================] - 1s 6ms/step - loss: 0.0562 - accura

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


90/90 [==============================] - 1s 8ms/step - loss: 1.3025 - accuracy: 0.8968 - val_loss: 0.3484 - val_accuracy: 0.8354 - lr: 0.1000
Epoch 2/200
90/90 [==============================] - 1s 6ms/step - loss: 0.1201 - accuracy: 0.9596 - val_loss: 0.2190 - val_accuracy: 0.9494 - lr: 0.1000
Epoch 3/200
90/90 [==============================] - 1s 6ms/step - loss: 0.0784 - accuracy: 0.9763 - val_loss: 0.1352 - val_accuracy: 0.9620 - lr: 0.0600
Epoch 4/200
90/90 [==============================] - 1s 6ms/step - loss: 0.0721 - accuracy: 0.9721 - val_loss: 0.1341 - val_accuracy: 0.9494 - lr: 0.0600
Epoch 5/200
90/90 [==============================] - 1s 6ms/step - loss: 0.0602 - accuracy: 0.9791 - val_loss: 0.1428 - val_accuracy: 0.9494 - lr: 0.0600
Epoch 6/200
90/90 [==============================] - 1s 6ms/step - loss: 0.0434 - accuracy: 0.9874 - val_loss: 0.1245 - val_accuracy: 0.9620 - lr: 0.0360
Epoch 7/200
90/90 [==============================] - 1s 6ms/step - loss: 0.0383 - accura

In [11]:
from statistics import mean, stdev
print(mean(ACC_collecton),'±',stdev(ACC_collecton))
print(mean(BACC_collecton),'±',stdev(BACC_collecton))
print(mean(Sn_collecton),'±',stdev(Sn_collecton))
print(mean(Sp_collecton),'±',stdev(Sp_collecton))
print(mean(MCC_collecton),'±',stdev(MCC_collecton))
print(mean(AUC_collecton),'±',stdev(AUC_collecton))


0.9760917721518987 ± 0.015069157059107196
0.9716010295561927 ± 0.026626543642595472
0.9652380952380952 ± 0.04979423521270552
0.97796396387429 ± 0.01463285092778108
0.9054583397942533 ± 0.05261340591987753
0.9780359548650902 ± 0.020790983784297903


### model evaluation in test dataset

In [12]:
# result collection list
ACC_collecton = []
BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []
model, model_history = ESM_CNN(X_train, y_train, X_test , y_test)
# confusion matrix 
predicted_class= []
predicted_protability = model.predict(X_test,batch_size=1)
for i in range(predicted_protability.shape[0]):
  index = np.where(predicted_protability[i] == np.amax(predicted_protability[i]))[0][0]
  predicted_class.append(index)
predicted_class = np.array(predicted_class)
y_true = y_test    
from sklearn.metrics import confusion_matrix
import math
# np.ravel() return a flatten 1D array
TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
ACC = (TP+TN)/(TP+TN+FP+FN)
ACC_collecton.append(ACC)
Sn_collecton.append(TP/(TP+FN))
Sp_collecton.append(TN/(TN+FP))
MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
MCC_collecton.append(MCC)
BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
from sklearn.metrics import roc_auc_score
AUC = roc_auc_score(y_test, predicted_protability[:,1])
AUC_collecton.append(AUC)

Epoch 1/200


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


100/100 [==============================] - 1s 7ms/step - loss: 1.1781 - accuracy: 0.8982 - val_loss: 0.2764 - val_accuracy: 0.9548 - lr: 0.1000
Epoch 2/200
100/100 [==============================] - 1s 6ms/step - loss: 0.1172 - accuracy: 0.9585 - val_loss: 0.1241 - val_accuracy: 0.9799 - lr: 0.1000
Epoch 3/200
100/100 [==============================] - 1s 6ms/step - loss: 0.0856 - accuracy: 0.9661 - val_loss: 0.0743 - val_accuracy: 0.9799 - lr: 0.0600
Epoch 4/200
100/100 [==============================] - 1s 6ms/step - loss: 0.0825 - accuracy: 0.9749 - val_loss: 0.0413 - val_accuracy: 0.9849 - lr: 0.0600
Epoch 5/200
100/100 [==============================] - 1s 7ms/step - loss: 0.0758 - accuracy: 0.9749 - val_loss: 0.1268 - val_accuracy: 0.9698 - lr: 0.0600
Epoch 6/200
100/100 [==============================] - 1s 6ms/step - loss: 0.0571 - accuracy: 0.9862 - val_loss: 0.0327 - val_accuracy: 0.9899 - lr: 0.0360
Epoch 7/200
100/100 [==============================] - 1s 6ms/step - loss: 0

In [13]:
print(ACC_collecton[0])
print(BACC_collecton[0])
print(Sn_collecton[0])
print(Sp_collecton[0])
print(MCC_collecton[0])
print(AUC_collecton[0])

0.9899497487437185
0.9941520467836258
1.0
0.9883040935672515
0.9604255067396437
0.9986193293885601


In [14]:
model.save('Anti_MRSA_tensorflow_model',save_format = 'tf') 
!zip -r /content/Anti_MRSA_tensorflow_model.zip /content/Anti_MRSA_tensorflow_model

  adding: content/Anti_MRSA_tensorflow_model/ (stored 0%)
  adding: content/Anti_MRSA_tensorflow_model/variables/ (stored 0%)
  adding: content/Anti_MRSA_tensorflow_model/variables/variables.index (deflated 64%)
  adding: content/Anti_MRSA_tensorflow_model/variables/variables.data-00000-of-00001 (deflated 42%)
  adding: content/Anti_MRSA_tensorflow_model/saved_model.pb (deflated 88%)
  adding: content/Anti_MRSA_tensorflow_model/assets/ (stored 0%)
  adding: content/Anti_MRSA_tensorflow_model/keras_metadata.pb (deflated 89%)
